In [ ]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def get_pdfsummary(reader):
    txt = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            txt += text

    return txt


PDF reader

In [ ]:
reader_1 = PdfReader("Krishnakant_bhutada_Data_Scientist_resume.pdf")
reader_2 = PdfReader("Krishnakant_Linkedin_profile.pdf")

summary = ""

summary += get_pdfsummary(reader_1)
summary += get_pdfsummary(reader_2)

# print(summary)

Tools

In [ ]:
def record_unknown_question(question):
    print(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

Main code

In [ ]:
name = "Krishnakant Bhutada"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s resume and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

# print(system_prompt)


In [ ]:
#Tool call 

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "record_unknown_question":
            print("reached here")
            result = record_unknown_question(**tool_args)
            print(result)
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    
    return results


In [ ]:
# Function to call OpenAI

def chat(message,history):

    messages=[{"role": "system", "content": system_prompt}]+ history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model="gpt-4.1-nano",   # or "gpt-4" if you have access
        messages=messages,
        tools = tools
    )
    
    finish_reason = response.choices[0].finish_reason

    print(finish_reason)
    print(response.choices[0].message.tool_calls)

    if finish_reason == "tool_calls":
        tool_calls = response.choices[0].message.tool_calls
        
        message = response.choices[0].message

        result  = handle_tool_calls(tool_calls)
        print(result)
        messages.append(message)
        messages.extend(result)
        return "✅ Tool call handled."
    

    return response.choices[0].message.content

    


In [ ]:
gr.ChatInterface(chat, type="messages").launch()